# Files Controleren met PathLib

In [89]:
from datetime import datetime
from pathlib import Path
import os
import re
import hashlib

import pandas as pd

pd.set_option('display.max_rows', 1000)

# from analyze_files import recursive_walk   # recursive_walk()
# import analyze_files                       # analyze_files.recursive_walk()
# from analyze_files import *                # recursive_walk()

In [90]:
def recursive_walk(directory: str, glob_pattern: str = None, re_pattern: str = None):
    """Recursively travels through directory and it's subdirectories.
    
    directory: directory to search
    glob_pattern: (optional) file matching string with wild cards
    re_pattern: (optional) file template string with regular expression

    The function is a generator returning a dict for each file.

    The function gracefully exits on KeyboardInterrupt (Control-C)
    
    Examples:
    
    >>> all_files = list(recursive_walk('..'))
    
    >>> all_files = list(recursive_walk('..', glob_pattern = '*.csv'))

    >>> all_files = list(recursive_walk('..', re_pattern = '.*ca-.*'))
    """

    try:
        if re_pattern:
            regex = re.compile(re_pattern)
        
        for i, (root, dirs, files) in enumerate(os.walk(directory)):
            if i % 1000 == 0:
                print(i, end='\r')
            
            dirs[:] = [d for d in dirs if not d.startswith('.')]
            files[:] = [f for f in files if f[0] not in ".~"]
            # if glob_pattern:
            #     files[:] = [f.name for f in root.glob(glob_pattern)]
            if re_pattern:
                files[:] = [f for f in files if regex.match(f)]
            
            # print(f"Directory path: {root}")
            # print(f"Directory Names: {dirs}")
            # print(f"Files Names: {files}")
            # print(80 * '-')
    
            for filename in files:
                file_path = os.path.join(root, filename)

                owner = None
                group = None
                if os.path.isfile(file_path):
                    path = Path(file_path)
                    owner = path.owner()
                    group = path.group()
    
                # Get file stats
                stat = os.stat(file_path, follow_symlinks=False)
                
                # File size in bytes
                size = stat.st_size
                
                # Creation time (platform dependent)
                ctime = datetime.fromtimestamp(stat.st_ctime)  # On Unix: metadata change time, on Windows: creation time
                
                # Modification time
                mtime = datetime.fromtimestamp(stat.st_mtime)
                
                # Last Access time
                atime = datetime.fromtimestamp(stat.st_atime)

                yield {
                    "filename": filename,
                    "directory": str(root),
                    "size": size,
                    "owner": owner,
                    "group": group,
                    "created": ctime,
                    "modified": mtime,          
                    "accessed": atime,
                    # "md5": md5,
                }

    except KeyboardInterrupt:
        return

### Do the walk

In [45]:
directory = '/Users/peter'

glob_pattern = ''    # or e.g. '*.csv'
re_pattern = ''      # or e.g. '.*ca-.*'

In [46]:
# all_files = list(recursive_walk(directory))

all_files = list(recursive_walk(directory, 
                                glob_pattern = glob_pattern, 
                                re_pattern = re_pattern))

289000

In [91]:
len(all_files)

1359839

### Put results in a Pandas DataFrame

In [67]:
df = pd.DataFrame(all_files)

In [68]:
df.shape

(1359839, 8)

### Find duplicated files base on 'size'

In [69]:
subset = ['size', 'created']
duplicates = df.duplicated(subset=subset, keep=False)
df.loc[duplicates, 'duplicated'] = df[subset].apply(tuple, axis=1)

In [70]:
df_duplicated = df[df['duplicated'].notna()].sort_values('duplicated')
df_duplicated.shape

(109140, 9)

In [78]:
df_duplicated.set_index('duplicated').sort_index().tail(100)

,filename,directory,size,owner,group,created,modified,accessed
duplicated,,,,,,,,
"(2676744, 2025-10-08 09:19:18.762080)",pydevd_cython.c,/Users/peter/Library/spyder-6/envs/spyder-runt...,2676744,peter,staff,2025-10-08 09:19:18.762080,2025-09-18 04:20:54.000000,2025-10-08 09:18:47.000000
"(2676744, 2025-10-08 09:19:18.762080)",pydevd_cython.c,/Users/peter/Library/spyder-6/pkgs/debugpy-1.8...,2676744,peter,staff,2025-10-08 09:19:18.762080,2025-09-18 04:20:54.000000,2025-10-08 09:18:47.000000
"(2697903, 2025-01-23 22:59:45.008884)",Assets.car,/Users/peter/Library/Developer/CoreSimulator/D...,2697903,peter,staff,2025-01-23 22:59:45.008884,2025-01-23 22:57:51.728689,2025-01-23 22:59:46.494936
"(2697903, 2025-01-23 22:59:45.008884)",Assets.car,/Users/peter/Library/Developer/CoreSimulator/D...,2697903,peter,staff,2025-01-23 22:59:45.008884,2025-01-23 22:57:51.728689,2025-01-23 22:59:46.494936
"(2744343, 2025-11-14 22:14:15.257136)",A0FEBDC9527CDECF0FC0F93285FD1E2F58EF64B8,/Users/peter/Library/Containers/51382381-85EE-...,2744343,peter,staff,2025-11-14 22:14:15.257136,2025-11-14 22:10:19.104254,2025-11-14 22:10:44.948014
"(2744343, 2025-11-14 22:14:15.257136)",3EEDBA876EB38B4BD69B088DCE8BABC910DB6144-blob,/Users/peter/Library/Containers/51382381-85EE-...,2744343,peter,staff,2025-11-14 22:14:15.257136,2025-11-14 22:10:19.104254,2025-11-14 22:10:44.948014
"(2799120, 2025-10-08 09:19:34.854527)",libQt5Qml.5.15.15.dylib,/Users/peter/Library/spyder-6/pkgs/qt-main-5.1...,2799120,peter,staff,2025-10-08 09:19:34.854527,2025-08-24 23:21:00.467454,2025-10-09 11:53:57.566755
"(2799120, 2025-10-08 09:19:34.854527)",libQt5Qml.5.15.15.dylib,/Users/peter/Library/spyder-6/envs/spyder-runt...,2799120,peter,staff,2025-10-08 09:19:34.854527,2025-08-24 23:21:00.467454,2025-10-09 11:53:57.566755
"(3172848, 2025-10-08 09:19:38.993546)",QtCore.abi3.so,/Users/peter/Library/spyder-6/envs/spyder-runt...,3172848,peter,staff,2025-10-08 09:19:38.993546,2025-06-06 18:20:45.343191,2025-10-09 11:49:18.397014


In [82]:
df_duplicated.groupby('duplicated').size()

duplicated
(0, 2025-01-06 10:40:16.987529)            5
(0, 2025-01-06 10:41:11.636501)            5
(0, 2025-01-06 10:41:21.969260)            5
(0, 2025-01-06 10:41:58.372670)            5
(0, 2025-01-23 21:36:54.241152)            5
                                          ..
(69453824, 2025-10-08 09:19:19.244218)     2
(101733024, 2025-10-08 09:19:39.411753)    2
(122226432, 2025-10-08 09:19:17.053791)    2
(140207680, 2025-10-08 09:19:17.055789)    2
(191046608, 2025-10-08 09:19:12.545386)    2
Length: 53430, dtype: int64

### Add md5 fingerprint

In [92]:
for index, row in df_duplicated.iterrows():
    file_path = os.path.join(row['directory'], row['filename'])

    # print(index, file_path)

    try:
        with open(file_path, mode = 'rb') as f:
            md5 = hashlib.md5(f.read()).hexdigest()
    except KeyboardInterrupt as ex:
        raise(ex)
    except:
        md5 = None
        
    df_duplicated.loc[index, 'md5'] = md5

In [100]:
df_duplicated.query('size>1024')[['directory','filename','size','created']].tail(500)

,directory,filename,size,created
1000708,/Users/peter/Library/spyder-6/envs/spyder-runt...,_ufuncs.pyx,559551,2025-10-08 09:19:38.417224
910092,/Users/peter/Library/spyder-6/pkgs/scipy-1.16....,_ufuncs.pyx,559551,2025-10-08 09:19:38.417224
988218,/Users/peter/Library/spyder-6/envs/spyder-runt...,momentsPen.c,565310,2025-10-08 09:19:22.773232
893637,/Users/peter/Library/spyder-6/pkgs/fonttools-4...,momentsPen.c,565310,2025-10-08 09:19:22.773232
997448,/Users/peter/Library/spyder-6/envs/spyder-runt...,test_axes.cpython-312.pyc,571248,2025-10-08 09:19:38.767259
908725,/Users/peter/Library/spyder-6/pkgs/matplotlib-...,test_axes.cpython-312.pyc,571248,2025-10-08 09:19:38.767259
891245,/Users/peter/Library/spyder-6/pkgs/spyder-base...,Main.js,572305,2025-10-08 09:19:40.728392
980741,/Users/peter/Library/spyder-6/envs/spyder-runt...,Main.js,572305,2025-10-08 09:19:40.728392
941461,/Users/peter/Library/spyder-6/pkgs/qt-webengin...,hi.pak,573839,2025-10-08 09:19:39.441522
1008266,/Users/peter/Library/spyder-6/envs/spyder-runt...,hi.pak,573839,2025-10-08 09:19:39.441522


### Validate filename format is YYYYMMDD_NU_filename.ext (NU, NR or NS)

In [10]:
regex = r'(\d{8})_(\w{2})_.*'

df[['filename_dd', 'filename_classificatie']] = df['filename'].str.extract(regex)

df['filename_ok'] = df['filename_dd'].notna() & \
                    pd.to_datetime(df['filename_dd'], format='%Y%m%d', errors='coerce').notna() & \
                    df['filename_classificatie'].notna() & \
                    df['filename_classificatie'].isin(['NU', 'NR', 'NS'])

In [11]:
df

,filename,directory,size,owner,group,created,modified,accessed,md5,duplicated,filename_dd,filename_classificatie,filename_ok
0,OneDrive,/Users/peter,352,peter,staff,2025-11-16 14:27:22.705283,2025-09-06 22:48:54.109681,2025-09-06 22:48:54.109854,None,"(352,)",NaN,NaN,False
1,attributed_ports.geojson,/Users/peter/Femke,236214,peter,staff,2025-02-05 08:49:07.554563,2023-10-15 21:27:20.425996,2023-10-15 21:27:20.439581,13887acf8e0817a39df5f1b4e45274a9,NaN,NaN,NaN,False
2,ports_distance.py,/Users/peter/Femke,2021,peter,staff,2024-01-26 13:57:49.827479,2023-11-09 23:49:11.322718,2025-11-13 17:07:11.950977,47f91df47de9b499692457ada09b8673,NaN,NaN,NaN,False
3,port_distances.pickle,/Users/peter/Femke,2293775612,peter,staff,2023-11-11 17:53:00.462749,2023-11-11 17:53:00.462749,2023-12-16 12:26:06.066918,78946eab414a0d8b757c28faf440c816,NaN,NaN,NaN,False
4,portdata.csv,/Users/peter/Femke/data,36994,peter,staff,2023-10-13 09:55:12.230644,2023-09-22 10:08:20.000000,2023-11-04 11:54:03.963699,704d33f6dfde84c0e349865159eefe93,"(36994,)",NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4230,Index.zip,/Users/peter/_Privé/Mark/CV Mark.pages,47787,peter,staff,2022-01-03 11:59:42.649269,2021-09-12 13:33:40.000000,2021-11-04 14:16:53.311504,200b954d81ed173d35f3339892d3bed6,NaN,NaN,NaN,False
4231,Hardcover_bullet_black-13.png,/Users/peter/_Privé/Mark/CV Mark.pages/Data,7007,peter,staff,2022-01-03 11:59:42.649368,2021-09-12 13:33:40.000000,2021-11-04 14:16:53.304724,75ba0de08067f948e87c44a4960cb4e4,NaN,NaN,NaN,False
4232,DocumentIdentifier,/Users/peter/_Privé/Mark/CV Mark.pages/Metadata,36,peter,staff,2022-01-03 11:59:42.649565,2021-09-12 13:33:40.000000,2021-11-04 14:16:53.313867,9d45a9a352d6e90140cd39110fb08a19,NaN,NaN,NaN,False
4233,BuildVersionHistory.plist,/Users/peter/_Privé/Mark/CV Mark.pages/Metadata,220,peter,staff,2022-01-03 11:59:42.649604,2021-09-12 13:33:40.000000,2021-11-04 14:16:53.313024,3bed317941b73bccaffabf9c1d8045e4,NaN,NaN,NaN,False
